# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [27]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = './data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [29]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [30]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.6960299128976762

In [31]:
# 空值補 0
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971535084032942

In [47]:
# 空值補平均值
df_mn = df.fillna(df.mean())
print(df.mean()) # 每一個col的mean都會算 Type: <class 'pandas.core.series.Series'>
train_X = df_mn[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

<class 'pandas.core.series.Series'>


0.6981761033723469

In [46]:
print(np.median(df[df.notnull()], axis=0))
print(df.median())# Type: <class 'pandas.core.series.Series'>
#========================
# 空值補中位數
df_med = df.fillna(df.median())
train_X = df_med[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

[ 3. nan  0.  0. nan]
Pclass     3.0000
Age       28.0000
SibSp      0.0000
Parch      0.0000
Fare      14.4542
dtype: float64
<class 'pandas.core.series.Series'>


0.6992934218081011

In [50]:
print(type(df.mode().iloc[0])) # <class 'pandas.core.series.Series'>
print(type(df.mode())) # <class 'pandas.core.frame.DataFrame'>
# 空值補眾數
df_mod = df.fillna(df.mode().iloc[0])
train_X = df_mod[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


0.6936879984197388

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [65]:
def Std_with_LogisticRegression(Fill,Encoder, df, title):
    df = df.fillna(Fill)
    if Encoder:
        df_temp = Encoder.fit_transform(df)
    else:
        df_temp = df
    train_X = df_temp[:train_num]
    estimator = LogisticRegression()
    print(title, ' : ',cross_val_score(estimator, train_X, train_Y, cv=5).mean())
#============

Std_with_LogisticRegression(-1, 0, df, '-1')
Std_with_LogisticRegression(-1, MinMaxScaler(), df, '-1+MinMaxScaler()')
Std_with_LogisticRegression(-1, StandardScaler(), df, '-1+StandardScaler()')

Std_with_LogisticRegression(0, 0, df, '0')
Std_with_LogisticRegression(0, MinMaxScaler(), df, '0+MinMaxScaler()')
Std_with_LogisticRegression(0, StandardScaler(), df, '0+StandardScaler()')

Std_with_LogisticRegression(df.median(), 0, df, 'median')
Std_with_LogisticRegression(df.median(), MinMaxScaler(), df, 'median+MinMaxScaler()')
Std_with_LogisticRegression(df.median(), StandardScaler(), df, 'median+StandardScaler()')

Std_with_LogisticRegression(df.mean(), 0, df, 'mean')
Std_with_LogisticRegression(df.mean(), MinMaxScaler(), df, 'mean+MinMaxScaler()')
Std_with_LogisticRegression(df.mean(), StandardScaler(), df, 'mean+StandardScaler()')

Std_with_LogisticRegression(df.mode().iloc[0], 0, df, 'mode')
Std_with_LogisticRegression(df.mode().iloc[0], MinMaxScaler(), df, 'mode+MinMaxScaler()')
Std_with_LogisticRegression(df.mode().iloc[0], StandardScaler(), df, 'mode+StandardScaler()')

-1  :  0.6960299128976762
-1+MinMaxScaler()  :  0.6971346062663598
-1+StandardScaler()  :  0.6982582017719778
0  :  0.6971535084032942
0+MinMaxScaler()  :  0.7016289882888318
0+StandardScaler()  :  0.6982582017719778
median  :  0.6992934218081011
median+MinMaxScaler()  :  0.6982328807104934
median+StandardScaler()  :  0.6970587140092496
mean  :  0.6981761033723469
mean+MinMaxScaler()  :  0.6993501991462476
mean+StandardScaler()  :  0.6959413955734954
mode  :  0.6936879984197388
mode+MinMaxScaler()  :  0.695998314766328
mode+StandardScaler()  :  0.7015532378864077


填補0 ＋ MinMaxScaler()最小最大化  :  0.7016289882888318  
效果最佳